# Сегментация временных рядов для автоматического прогнозирования 
Автор работы: Сентемова Ольга

Научный руководитель: Алексей Романенко

# Постановка задачи

В работе представлен алгоритм для прогнозированния семейства временных рядов с помощью классических методов. В качестве такого семейства может выступать группа временных рядов описываюших продажи различных товаров в магазинах сетей. Это типичная задача, возникающая в ритейл индустрии при планировании спроса на конкретный товар и объема закупок для конкретного магазина. 

Для прогнозирования, все временные ряды будут разбиты на классы в зависимости от параметров конкретного временного ряда. Алгоритм сегментации основывается на внутренних характеристиках временного ряда, такие как стохастичность и сезонность. Эти параметры определяются динамически. 

Для каждого класса будет подобрано наиболее подходящее семейство моделей для  прогнозирования. Внутри семейства будут подобраны параметры, которые будут предлагать прогнозирование лучшего качества. 

## Используемые данные

В данной работе для демонстрации подхода будет рассматриваться датасет, предоставленный для соревнования [M5 Forecasting Accuracy](https://www.kaggle.com/competitions/m5-forecasting-accuracy/data#). Это типичный датасет для ритейл-индустрии, он может быть ярким примером семейства данных, требующих прогнозирования. 